In [13]:
### Library Imports
import pandas as pd
import yfinance as yf
import numpy as np


In [14]:
### Function to Import Stock Data
def import_stock_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date, end_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date, end_date)['Adj Close']
    
    # Reset index to include the Date as a column
    data = data.reset_index()

    return data

tickers = ['AAPL', 'MSFT', 'GOOG', 'JNJ', 'XOM', 'TSLA', 'JPM', 'UNH', 'NVDA', 'PG']
start_date = '2019-01-01'
end_date = '2024-01-01'
stock_data = import_stock_data(tickers, start_date, end_date)
print(stock_data.tail())

[*********************100%%**********************]  10 of 10 completed


           Date        AAPL        MSFT        GOOG         JNJ         XOM  \
1253 2023-12-22  192.868134  142.557770  151.898438  164.497726  372.543945   
1254 2023-12-26  192.320221  142.657669  152.562851  165.470566  372.623505   
1255 2023-12-27  192.419830  141.279236  152.768051  166.463058  372.036713   
1256 2023-12-28  192.848206  141.119415  152.992767  167.347473  373.240112   
1257 2023-12-29  191.802170  140.769806  153.149109  167.150925  373.996002   

           TSLA         JPM         UNH        NVDA         PG  
1253  48.823704  142.584473  252.539993  516.077576  99.358162  
1254  49.272640  143.232239  256.609985  515.799866  99.582405  
1255  49.410622  143.350006  261.440002  518.537415  99.114418  
1256  49.515610  143.026123  253.179993  520.630249  97.681229  
1257  49.515610  143.821106  248.479996  522.187439  97.476494  


In [18]:
### Function to Compute Daily Returns
''' 
The percentage change in stock price from day t-1 to day t is given by:
    R(i,t) = [P(i,t) - P(i,t-1)]​ / P(i,t-1)
In our case, use the pre-built python pct_change() function
'''
def daily_returns(stock_data):
    # Drop the date column
    stock_data = stock_data.drop(columns = ['Date'])
    # Compute the percentage change
    rets = stock_data.pct_change().dropna()
    
    return rets

# Call the function to compute the daily returns for error checking
returns = daily_returns(stock_data)
print(returns.tail())

### Function to Compute Volatilities
def volatility(tickers, returns):
    # Create list to store volatilities
    volatilities = []  
    # Iterate over each ticker in tickers list
    for ticker in tickers:
        # Compute annualized volatility
        sigma = np.std(returns[ticker]) * np.sqrt(252)  
        volatilities.append(round(sigma, 5))  
        
    return volatilities

# Call the function to compute the volatility for each stock
vols = volatility(tickers, returns)
print(vols)


          AAPL      MSFT      GOOG       JNJ       XOM      TSLA       JPM  \
1253 -0.005547  0.006488  0.004004 -0.000597  0.002784 -0.003266  0.007071   
1254 -0.002841  0.000701  0.004374  0.005914  0.000214  0.009195  0.004543   
1255  0.000518 -0.009663  0.001345  0.005998 -0.001575  0.002800  0.000822   
1256  0.002226 -0.001131  0.001471  0.005313  0.003235  0.002125 -0.002259   
1257 -0.005424 -0.002477  0.001022 -0.001174  0.002025  0.000000  0.005558   

           UNH      NVDA        PG  
1253 -0.007701  0.000827  0.001769  
1254  0.016116 -0.000538  0.002257  
1255  0.018822  0.005307 -0.004699  
1256 -0.031594  0.004036 -0.014460  
1257 -0.018564  0.002991 -0.002096  
[0.32222, 0.31806, 0.19855, 0.31885, 0.30479, 0.51749, 0.21153, 0.64674, 0.29493, 0.34262]


In [28]:
### Function to build Risk-Parity Portfolio
''' 
The Risk-Parity Portfolio method allocates more weight to less volatile stocks and less weight to more volatile stocks, 
aiming to equalize the contribution of each stock to the total portfolio risk. Calculation is as follows:
    1. Estimate the volatility (standard deviation) of each stock's returns
    2. Assign weights inversely proportional to the volatility:
        w(i) = (1/σ(i)) / (sum_(j=1)^10 1/σ(j))
'''
def risk_parity_weights(volatilities):
    # Empty list to store weights
    weights = []
    # Inverse sum volatilities for denominator
    inv_sum_vols = sum(1 / vol for vol in volatilities)
    # Iterate over values in volatilities list
    for vol in volatilities:
        # Weights calculation
        weight = (1 / vol) / inv_sum_vols
        weights.append(round(weight, 5))

    return weights

weights = risk_parity_weights(vols)
print('The sum of the weights is:', round(sum(weights), 6), 'and should be 1')
print(weights)


The sum of the weights is: 0.99999 and should be 1
[0.09633, 0.09759, 0.15633, 0.09735, 0.10184, 0.05998, 0.14674, 0.04799, 0.10524, 0.0906]


In [32]:
### Compute Daily Portfolio Returns
''' 
The daily portfolio return is computed as the weighted sum of the returns of each stock for that particular day, equation:
   R_{portfolio,t} = (sum_(i=1)^10 w_i * R_{i,t})

For Each Day t (e.g., Day 1):
    Take the return of stock 1 on day 1 and multiply it by the weight of stock 1.
    Take the return of stock 2 on day 1 and multiply it by the weight of stock 2.
    Repeat this for all 10 stocks.

Then, sum all these weighted returns to get the portfolio return for day 1:
    R_{portfolio,t=1} = w_1R_{1,1} + w_2R_{2,1} +...+ w_10R_{10,1}
'''
# Compute the portfolio returns by multiplying the daily returns with the weights
portfolio_rets = returns.dot(weights)
# Convert portfolio rets to df
dates = stock_data['Date']
portfolio_rets_df = pd.DataFrame({
    'Date': dates,
    'Portfolio_Returns': portfolio_rets
})
# Display the portfolio returns
print(portfolio_rets_df.tail())


           Date  Portfolio_Returns
1253 2023-12-22           0.001670
1254 2023-12-26           0.003215
1255 2023-12-27           0.001065
1256 2023-12-28          -0.001425
1257 2023-12-29          -0.000463


In [34]:
### Build Variance-Covariance Matrix
''' 
The covariance between two stocks i and j is calculated as:
    Cov(R_i, R_j) = (1 / (T - 1)) * sum_{t=1}^T (R_{i,t} - R̄_i)(R_{j,t} - R̄_j)
Where:
    R_{i,t} and R_{j,t} are the daily returns of stocks i and j on day t,
    R̄_i and R̄_j are the average returns of stocks i and j over the period,
    T is the number of time periods (e.g., 1,257 days in your case).
'''
def var_cov_matrix(daily_rets):
    # Find T (number of days)
    T = len(returns)
    




cov_matrix = var_cov_matrix(returns)

1257
